# Black-Litterman Portfolio's Theory #

### Practical Example ###

In [239]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Optiminization
from scipy.optimize import minimize

# Pretty Notation
from IPython.display import display, Math

In [240]:
def import_financial_data(
    ticker: str
):

    # Check the ticker for Upper Cases
    ticker = ticker if ticker.isupper() else ticker.upper()

    # Import data
    df = pd.read_csv(rf"..\stocks\{ticker}.csv")

    # Set the Index
    df = df.set_index('Date')
    df.index = pd.to_datetime(df.index)

    df_useful_data = df[['Open Price', 'High Price', 'Low Price', 'Close Price', 'Adjusted_close', 'Company Market Cap']]

    df_useful_data = df_useful_data.rename(columns={
        "Open Price":"open",
        "High Price":"high",
        "Low Price":"low",
        "Close Price":"close",
        "Adjusted_close":"adjusted_close",
        "Company Market Cap":"mkt_cap"
    })

    # Drop NaN's
    df_useful_data.dropna(inplace = True)

    return df_useful_data.loc["2023-01-01":]

In [241]:
# Import Data

# Apple Data
df_1 = import_financial_data("AAPL")

# Amazon Data
df_2 =  import_financial_data("AMZN")

# Meta Data
df_3 =  import_financial_data("META")

# Microsoft Data
df_4 =  import_financial_data("MSFT")

In [242]:
# Create the joint dataframe
df_data = pd.DataFrame()

# Columns will be the Adjusted Close Price
df_data['AAPL'] = df_1['adjusted_close']
df_data['AMZN'] = df_2['adjusted_close']
df_data['META'] = df_3['adjusted_close']
df_data['MSFT'] = df_4['adjusted_close']

# Drop Missing Data
df_data = df_data.dropna()

df_returns = df_data.pct_change(1).mul(100)
df_returns = df_returns.apply(lambda x: x.fillna(x.mean()), axis=0)

df_returns

In [243]:
# Theoretically we could use the average as the expected returns (these are daily returns)
expected_returns = df_returns.mean() 

expected_returns

In [244]:
# The volatility is calculated with the standard deviations (also daily volatilities)
volat = df_returns.dropna().std() 

volat

In [245]:
# Covariance Matrix
cov_matrix = df_returns.dropna().cov()

cov_matrix

In [246]:
# Create the Market Capitalization 
df_mktcap = pd.DataFrame()

df_mktcap['AAPL'] = df_1['mkt_cap']
df_mktcap['AMZN'] = df_2['mkt_cap']
df_mktcap['META'] = df_3['mkt_cap']
df_mktcap['MSFT'] = df_4['mkt_cap']

df_mktcap = df_mktcap.dropna()

df_mktcap

In [247]:
# Create the Market Capitalization Weights

total_market_cap = df_mktcap.sum(axis=1)  # Horizontal Sum because we got a Time Series

df_mktcap_weights = df_mktcap.div(total_market_cap, axis = 0)

df_mktcap_weights

In [248]:
# Let us use the last observations since the mean of the portfolio is calculated with the whole story

market_weights = df_mktcap_weights.iloc[-1]

market_weights

In [249]:
# We can calculate or estimate the Risk Aversion Coefficient using market data, but let us assume it

risk_aversion = 3.0

In [250]:
# Compute implied equilibrium returns

pi = risk_aversion * cov_matrix @ market_weights

pi

In [251]:
# Tau adjustment (controls uncertainty of implied returns)
tau = 0.10

pi_adjusted = tau * pi

pi_adjusted

In [252]:
# P matrix: 1 view per asset (identity matrix)
P = np.identity(4)

# Q vector: our expected returns relative to the market
Q = np.array([0.01, 0.02, 0.05, 0.03])  # AAPL, AMZN, META, MSFT

In [253]:
# Diagonal Omega matrix: uncertainty of each view
# Often proportional to variance of the assets related to the views
Omega = np.diag(np.diag(P @ cov_matrix @ P.T)) * tau

Omega

In [254]:
# Inverse Matrix

inv_tau_sigma = np.linalg.inv(tau * cov_matrix)
inv_omega = np.linalg.inv(Omega)

In [255]:
# Apply Black-Litterman formula

posterior_mean = np.linalg.inv(inv_tau_sigma + P.T @ inv_omega @ P) @ (inv_tau_sigma @ pi + P.T @ inv_omega @ Q)

posterior_mean

In [256]:
# Calculate the Optimal Weights

optimal_weights = np.linalg.inv(cov_matrix) @ (posterior_mean) / risk_aversion

# Normalize weights to sum to 1

optimal_weights /= np.sum(optimal_weights)

optimal_weights

In [257]:
# Calculate the Portfolio Returns and Variance

portfolio_returns = np.dot(optimal_weights, expected_returns)

portfolio_variance = np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights))

portfolio_volatility = np.sqrt(portfolio_variance)

print(f"Expected Portfolio Return: {portfolio_returns:.4f}")
print(f"Portfolio Variance: {portfolio_variance:.6f}")
print(f"Portfolio Volatility (Std Dev): {portfolio_volatility:.4f}")

In [258]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_data['AAPL'], label='APPL Price', color='blue', alpha=0.7)
plt.plot(df_data['AMZN'], label='AMZN Price', color='green', alpha=0.7)
plt.plot(df_data['META'], label='META Price', color='red', alpha=0.7)
plt.plot(df_data['MSFT'], label='MSFT Price', color='orange', alpha=0.7)

# Config
plt.title('Prices Time Series')
plt.xlabel('Time Index')
plt.ylabel('$P_t$')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [259]:
# Calculate the Historical Returns of the Portfolio

portfolio_returns = df_returns @ optimal_weights

portfolio_returns

In [260]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_returns['AAPL'].cumsum(), label='APPL Price', color='blue', alpha=0.7)
plt.plot(df_returns['AMZN'].cumsum(), label='AMZN Price', color='green', alpha=0.7)
plt.plot(df_returns['META'].cumsum(), label='META Price', color='red', alpha=0.7)
plt.plot(df_returns['MSFT'].cumsum(), label='MSFT Price', color='orange', alpha=0.7)
plt.plot(portfolio_returns.cumsum(), label='BL Portfolio', color='purple', alpha=0.7)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [261]:
def calculate_analytics(df_returns, risk_free_rate=0.0):
    # Trading Days in one Year
    ann_factor = 252  
    
    # Annualized Returns
    annualized_return = df_returns.mean() * ann_factor
    
    # Annualized Volatility
    annualized_std = df_returns.std() * np.sqrt(ann_factor)
    
    # Sharpe Ratio
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_std
    
    # Max Drawdown
    cumulative_returns = (1 + df_returns.div(100)).cumprod()
    rolling_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns / rolling_max) - 1
    max_drawdown = drawdown.min()

    # VaR at 95%
    var_95 = df_returns.quantile(0.05)

    # Create DF
    summary_df = pd.DataFrame({
        "Annualized Returns": annualized_return,
        "Annualized Volatility": annualized_std,
        "Sharpe Ratio": sharpe_ratio,
        "Max Drawdown": max_drawdown,
        "VaR 95%": var_95
    })
    
    return summary_df

In [262]:
# Create the Analytics

df_returns['BL'] = portfolio_returns

df_returns

In [263]:
# Now the table
analytics_table = calculate_analytics(df_returns)

analytics_table